In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 폰트 설정 (에러 방지)
plt.rcParams['axes.unicode_minus'] = False

class KepcoDataPreprocessor:
    """
    한국전력공사 LP 데이터 전처리 및 탐색적 분석
    
    단계별 분석 계획:
    1단계: 데이터 품질 점검 (30분)
    2단계: 기본 패턴 탐색 (60분) 
    3단계: 변동성 기초 분석 (90분)
    4단계: 이상 패턴 탐지 (60분)
    5단계: 전처리 방향 결정 (30분)
    """
    
    def __init__(self):
        self.data_quality_report = {}
        self.pattern_analysis = {}
        self.variability_analysis = {}
    
    # ============ 데이터 로딩 및 결합 ============
    
    def load_and_combine_lp_data(self, lp_files):
        """
        LP 데이터 파일들을 읽어서 결합
        파일들은 한 달을 반으로 나눠서 제공됨 (예: LP데이터1.csv + LP데이터2.csv)
        """
        print("📂 LP 데이터 파일들 로딩 및 결합 중...")
        
        combined_data = []
        
        for i, file_path in enumerate(lp_files):
            try:
                df = pd.read_csv(file_path)
                print(f"  ✅ {file_path}: {len(df):,}건 로딩")
                
                # 기본 정보 출력
                if 'LP수신일자' in df.columns:
                    dates = pd.to_datetime(df['LP수신일자'])
                    print(f"     기간: {dates.min()} ~ {dates.max()}")
                    print(f"     고객수: {df['대체고객번호'].nunique()}명")
                
                combined_data.append(df)
                
            except Exception as e:
                print(f"  ❌ {file_path} 로딩 실패: {e}")
        
        if not combined_data:
            raise ValueError("로딩된 LP 데이터가 없습니다.")
        
        # 데이터 결합
        final_data = pd.concat(combined_data, ignore_index=True)
        
        print(f"  🔗 결합 완료: 총 {len(final_data):,}건")
        print(f"     전체 기간: {pd.to_datetime(final_data['LP수신일자']).min()} ~ {pd.to_datetime(final_data['LP수신일자']).max()}")
        print(f"     총 고객수: {final_data['대체고객번호'].nunique()}명")
        
        return final_data
    
    # ============ 1단계: 데이터 품질 점검 (30분) ============
    
    def check_data_quality(self, lp_data, customer_data):
        """
        데이터 품질 점검 및 기본 정보 분석
        """
        print("🔍 1단계: 데이터 품질 점검 시작...")
        
        # 고객 기본정보 분석
        customer_info = self._analyze_customer_info(customer_data)
        
        # LP 데이터 품질 점검
        lp_quality = self._check_lp_data_quality(lp_data)
        
        self.data_quality_report = {
            'customer_info': customer_info,
            'lp_quality': lp_quality,
            'data_completeness': self._calculate_completeness(lp_data),
            'anomaly_detection': self._detect_data_anomalies(lp_data)
        }
        
        self._print_quality_summary()
        return self.data_quality_report
    
    def _analyze_customer_info(self, customer_data):
        """고객 기본정보 분석"""
        if customer_data is None:
            return {"message": "고객 데이터 없음"}
        
        print(f"  📋 고객 데이터 컬럼: {list(customer_data.columns)}")
        
        # 가능한 컬럼명들 매핑
        column_mapping = {
            '계약종별': ['계약종별', 'contract_type', 'Contract_Type'],
            '사용용도': ['사용용도', 'usage_purpose', 'Usage_Purpose'], 
            '산업분류': ['산업분류', 'industry', 'Industry']
        }
        
        info = {'total_customers': len(customer_data)}
        
        for key, possible_cols in column_mapping.items():
            found_col = None
            for col in possible_cols:
                if col in customer_data.columns:
                    found_col = col
                    break
            
            if found_col:
                info[f'{key}_dist'] = customer_data[found_col].value_counts().to_dict()
                print(f"  ✅ {key} 분포: {dict(list(info[f'{key}_dist'].items())[:3])}...")  # 상위 3개만 출력
            else:
                info[f'{key}_dist'] = {}
                print(f"  ⚠️ {key} 컬럼 없음")
        
        print(f"✅ 고객수: {info['total_customers']:,}명")
        return info
    
    def _check_lp_data_quality(self, lp_data):
        """LP 데이터 품질 점검"""
        # 데이터 타입 변환
        lp_data['LP수신일자'] = pd.to_datetime(lp_data['LP수신일자'])
        
        quality = {
            'total_records': len(lp_data),
            'date_range': {
                'start': lp_data['LP수신일자'].min(),
                'end': lp_data['LP수신일자'].max()
            },
            'unique_customers': lp_data['대체고객번호'].nunique(),
            'missing_values': lp_data.isnull().sum().to_dict(),
            'negative_values': (lp_data['순방향유효전력'] < 0).sum(),
            'zero_values': (lp_data['순방향유효전력'] == 0).sum(),
        }
        
        print(f"✅ 총 레코드: {quality['total_records']:,}건")
        print(f"✅ 기간: {quality['date_range']['start']} ~ {quality['date_range']['end']}")
        print(f"✅ 고객수: {quality['unique_customers']:,}명")
        print(f"✅ 음수값: {quality['negative_values']:,}건")
        print(f"✅ 0값: {quality['zero_values']:,}건")
        
        return quality
    
    def _calculate_completeness(self, lp_data):
        """데이터 완정성 계산"""
        lp_data['date'] = lp_data['LP수신일자'].dt.date
        lp_data['hour'] = lp_data['LP수신일자'].dt.hour
        lp_data['quarter_hour'] = (lp_data['LP수신일자'].dt.minute // 15) * 15
        
        # 15분 간격 정확성 체크
        expected_intervals = pd.date_range(
            start=lp_data['LP수신일자'].min(),
            end=lp_data['LP수신일자'].max(),
            freq='15min'
        )
        
        completeness = {
            'expected_records': len(expected_intervals) * lp_data['대체고객번호'].nunique(),
            'actual_records': len(lp_data),
            'completeness_rate': len(lp_data) / (len(expected_intervals) * lp_data['대체고객번호'].nunique()) * 100
        }
        
        # 고객별 완정성
        customer_completeness = lp_data.groupby('대체고객번호').size() / len(expected_intervals) * 100
        completeness['customer_completeness'] = {
            'mean': customer_completeness.mean(),
            'min': customer_completeness.min(),
            'max': customer_completeness.max(),
            'std': customer_completeness.std()
        }
        
        print(f"✅ 데이터 완정성: {completeness['completeness_rate']:.2f}%")
        
        return completeness
    
    def _detect_data_anomalies(self, lp_data):
        """데이터 이상치 탐지"""
        # 통계적 이상치 탐지
        Q1 = lp_data['순방향유효전력'].quantile(0.25)
        Q3 = lp_data['순방향유효전력'].quantile(0.75)
        IQR = Q3 - Q1
        
        outliers_iqr = lp_data[
            (lp_data['순방향유효전력'] < Q1 - 1.5 * IQR) | 
            (lp_data['순방향유효전력'] > Q3 + 1.5 * IQR)
        ]
        
        # Z-score 이상치
        z_scores = np.abs((lp_data['순방향유효전력'] - lp_data['순방향유효전력'].mean()) / lp_data['순방향유효전력'].std())
        outliers_zscore = lp_data[z_scores > 3]
        
        anomalies = {
            'iqr_outliers': len(outliers_iqr),
            'zscore_outliers': len(outliers_zscore),
            'outlier_rate_iqr': len(outliers_iqr) / len(lp_data) * 100,
            'outlier_rate_zscore': len(outliers_zscore) / len(lp_data) * 100
        }
        
        print(f"✅ IQR 이상치: {anomalies['outlier_rate_iqr']:.3f}%")
        print(f"✅ Z-score 이상치: {anomalies['outlier_rate_zscore']:.3f}%")
        
        return anomalies
    
    def _print_quality_summary(self):
        """품질 점검 요약 출력"""
        print("\n" + "="*50)
        print("📊 데이터 품질 점검 완료")
        print("="*50)
    
    # ============ 2단계: 기본 패턴 탐색 (60분) ============
    
    def analyze_basic_patterns(self, lp_data, customer_data=None, calendar_data=None):
        """
        기본 전력 사용 패턴 분석
        """
        print("\n📊 2단계: 기본 패턴 탐색 시작...")
        
        # 데이터 전처리
        processed_data = self._preprocess_for_pattern_analysis(lp_data, customer_data, calendar_data)
        
        # 시간별 패턴 분석
        time_patterns = self._analyze_time_patterns(processed_data)
        
        # 고객 세분화 기초 분석
        customer_segmentation = self._analyze_customer_segmentation(processed_data)
        
        self.pattern_analysis = {
            'time_patterns': time_patterns,
            'customer_segmentation': customer_segmentation,
            'processed_data': processed_data
        }
        
        return self.pattern_analysis
    
    def _preprocess_for_pattern_analysis(self, lp_data, customer_data, calendar_data):
        """패턴 분석을 위한 데이터 전처리"""
        print("  🔄 데이터 전처리 중...")
        
        # 시간 변수 생성
        lp_data['datetime'] = pd.to_datetime(lp_data['LP수신일자'])
        lp_data['date'] = lp_data['datetime'].dt.date
        lp_data['hour'] = lp_data['datetime'].dt.hour
        lp_data['weekday'] = lp_data['datetime'].dt.weekday  # 0=월요일
        lp_data['month'] = lp_data['datetime'].dt.month
        lp_data['quarter'] = lp_data['datetime'].dt.quarter
        lp_data['is_weekend'] = lp_data['weekday'].isin([5, 6])  # 토, 일
        
        # 일간 집계 데이터 생성
        daily_agg = lp_data.groupby(['대체고객번호', 'date']).agg({
            '순방향유효전력': ['sum', 'mean', 'max', 'min', 'std']
        }).reset_index()
        
        # 컬럼명 정리
        daily_agg.columns = ['customer_id', 'date', 'daily_sum', 'daily_mean', 'daily_max', 'daily_min', 'daily_std']
        
        # 날짜 관련 피처 다시 생성 (일간 집계 후)
        daily_agg['date_dt'] = pd.to_datetime(daily_agg['date'])
        daily_agg['weekday'] = daily_agg['date_dt'].dt.weekday  # 0=월요일
        daily_agg['month'] = daily_agg['date_dt'].dt.month
        daily_agg['quarter'] = daily_agg['date_dt'].dt.quarter
        daily_agg['is_weekend'] = daily_agg['weekday'].isin([5, 6])  # 토, 일
        
        print(f"  ✅ 시간 피처 생성 완료: weekday, month, quarter, is_weekend")
        
        # 고객 정보 병합
        if customer_data is not None:
            # 고객 데이터의 실제 컬럼명 확인
            customer_key_col = None
            possible_keys = ['대체고객번호', '고객번호', 'customer_id', 'Customer_ID']
            
            for col in possible_keys:
                if col in customer_data.columns:
                    customer_key_col = col
                    break
            
            if customer_key_col:
                daily_agg = daily_agg.merge(customer_data, left_on='customer_id', right_on=customer_key_col, how='left')
                print(f"  ✅ 고객 정보 병합 완료 (키: {customer_key_col})")
            else:
                print(f"  ⚠️ 고객 데이터 병합 실패 - 키 컬럼을 찾을 수 없음. 사용 가능한 컬럼: {list(customer_data.columns)}")
                print(f"  ℹ️ 고객 정보 없이 분석 계속...")
        
        # 기상/달력 정보 병합
        if calendar_data is not None:
            # 날짜 컬럼 확인 및 변환
            date_col = None
            possible_date_cols = ['date', '날짜', 'Date', 'DATE']
            
            for col in possible_date_cols:
                if col in calendar_data.columns:
                    date_col = col
                    break
            
            if date_col:
                # 기상 데이터가 weather_daily_processed.csv인 경우 날짜 형식 확인
                if 'year' in calendar_data.columns and 'month' in calendar_data.columns and 'day' in calendar_data.columns:
                    # year, month, day 컬럼으로 날짜 생성
                    calendar_data['date_parsed'] = pd.to_datetime(calendar_data[['year', 'month', 'day']])
                    calendar_data['date_for_merge'] = calendar_data['date_parsed'].dt.date
                    daily_agg = daily_agg.merge(calendar_data, left_on='date', right_on='date_for_merge', how='left')
                    print(f"  ✅ 기상/달력 정보 병합 완료 (year-month-day 기준)")
                else:
                    # 일반적인 date 컬럼 사용
                    calendar_data[date_col] = pd.to_datetime(calendar_data[date_col]).dt.date
                    daily_agg = daily_agg.merge(calendar_data, left_on='date', right_on=date_col, how='left')
                    print(f"  ✅ 기상/달력 정보 병합 완료 (키: {date_col})")
            else:
                print(f"  ⚠️ 기상/달력 데이터 병합 실패 - 날짜 컬럼을 찾을 수 없음. 사용 가능한 컬럼: {list(calendar_data.columns)}")
                print(f"  ℹ️ 기상/달력 정보 없이 분석 계속...")
        
        print(f"  ✅ 일간 집계 데이터: {len(daily_agg):,}건")
        return daily_agg
    
    def _analyze_time_patterns(self, data):
        """시간별 패턴 분석"""
        print("  📈 시간별 패턴 분석 중...")
        
        patterns = {}
        
        # 1. 시간대별 패턴 (일간 집계 데이터에서는 의미 없으므로 건너뛰기)
        # hourly_pattern = data.groupby('hour')['daily_mean'].agg(['mean', 'std', 'count'])
        # patterns['hourly'] = hourly_pattern
        
        # 2. 요일별 패턴
        if 'weekday' in data.columns:
            weekday_pattern = data.groupby('weekday')['daily_mean'].agg(['mean', 'std', 'count'])
            patterns['weekday'] = weekday_pattern
        
        # 3. 월별 패턴 (계절성)
        if 'month' in data.columns:
            monthly_pattern = data.groupby('month')['daily_mean'].agg(['mean', 'std', 'count'])
            patterns['monthly'] = monthly_pattern
        
        # 4. 주중/주말 패턴
        if 'is_weekend' in data.columns:
            weekend_pattern = data.groupby('is_weekend')['daily_mean'].agg(['mean', 'std', 'count'])
            patterns['weekend'] = weekend_pattern
        
        # 5. 업종별 패턴 (고객 데이터가 있는 경우)
        usage_purpose_col = None
        possible_usage_cols = ['사용용도', 'usage_purpose', 'Usage_Purpose']
        
        for col in possible_usage_cols:
            if col in data.columns:
                usage_purpose_col = col
                break
        
        if usage_purpose_col:
            industry_pattern = data.groupby(usage_purpose_col)['daily_mean'].agg(['mean', 'std', 'count'])
            patterns['industry'] = industry_pattern
            print(f"  ✅ {usage_purpose_col} 기준 업종별 패턴 분석 완료")
        
        print(f"  ✅ 시간별 패턴 분석 완료 ({len(patterns)}개 패턴)")
        return patterns
    
    def _analyze_customer_segmentation(self, data):
        """고객 세분화 기초 분석"""
        print("  👥 고객 세분화 분석 중...")
        
        # 고객별 평균 사용량 계산
        customer_avg = data.groupby('customer_id')['daily_mean'].mean()
        
        # 사용량 규모별 분류
        segmentation = {
            'large_users': customer_avg.quantile(0.9),  # 상위 10%
            'medium_users': customer_avg.quantile(0.5),  # 중간 50%
            'small_users': customer_avg.quantile(0.1),   # 하위 10%
        }
        
        # 고객별 사용량 분포
        customer_stats = {
            'customer_count': len(customer_avg),
            'usage_distribution': {
                'mean': customer_avg.mean(),
                'std': customer_avg.std(),
                'min': customer_avg.min(),
                'max': customer_avg.max(),
                'q25': customer_avg.quantile(0.25),
                'q50': customer_avg.quantile(0.50),
                'q75': customer_avg.quantile(0.75)
            },
            'segmentation_thresholds': segmentation
        }
        
        print(f"  ✅ {customer_stats['customer_count']:,}명 고객 세분화 완료")
        return customer_stats
    
    # ============ 3단계: 변동성 기초 분석 (90분) ============
    
    def analyze_variability(self, processed_data):
        """
        변동성 기초 분석 - 변동계수 설계를 위한 기초 작업
        """
        print("\n📈 3단계: 변동성 기초 분석 시작...")
        
        # 기본 변동성 지표 계산
        basic_variability = self._calculate_basic_variability(processed_data)
        
        # 변동성 패턴 분석
        variability_patterns = self._analyze_variability_patterns(processed_data)
        
        self.variability_analysis = {
            'basic_variability': basic_variability,
            'variability_patterns': variability_patterns
        }
        
        return self.variability_analysis
    
    def _calculate_basic_variability(self, data):
        """기본 변동성 지표 계산"""
        print("  📊 기본 변동성 지표 계산 중...")
        
        variability_metrics = {}
        
        # 고객별 변동계수 계산
        customer_cv = data.groupby('customer_id').apply(
            lambda x: x['daily_mean'].std() / x['daily_mean'].mean() if x['daily_mean'].mean() > 0 else np.nan
        )
        
        # 1. 일간 변동계수
        variability_metrics['daily_cv'] = {
            'mean': customer_cv.mean(),
            'std': customer_cv.std(),
            'distribution': customer_cv.describe()
        }
        
        # 2. 주간 변동계수 (주별 패턴의 일관성)
        try:
            # 주 번호 생성
            data_with_week = data.copy()
            data_with_week['week'] = pd.to_datetime(data_with_week['date']).dt.isocalendar().week
            
            weekly_cv = data_with_week.groupby(['customer_id', 'week']).agg({
                'daily_mean': ['mean', 'std']
            }).reset_index()
            weekly_cv.columns = ['customer_id', 'week', 'weekly_mean', 'weekly_std']
            weekly_cv['weekly_cv'] = weekly_cv['weekly_std'] / weekly_cv['weekly_mean']
            
            customer_weekly_cv = weekly_cv.groupby('customer_id')['weekly_cv'].mean()
            variability_metrics['weekly_cv'] = {
                'mean': customer_weekly_cv.mean(),
                'std': customer_weekly_cv.std(),
                'distribution': customer_weekly_cv.describe()
            }
        except Exception as e:
            print(f"    ⚠️ 주간 변동계수 계산 실패: {e}")
            variability_metrics['weekly_cv'] = {'mean': np.nan, 'std': np.nan}
        
        # 3. 월간 변동계수
        try:
            if 'month' in data.columns:
                monthly_cv = data.groupby(['customer_id', 'month']).agg({
                    'daily_mean': ['mean', 'std']
                }).reset_index()
                monthly_cv.columns = ['customer_id', 'month', 'monthly_mean', 'monthly_std']
                monthly_cv['monthly_cv'] = monthly_cv['monthly_std'] / monthly_cv['monthly_mean']
                
                customer_monthly_cv = monthly_cv.groupby('customer_id')['monthly_cv'].mean()
                variability_metrics['monthly_cv'] = {
                    'mean': customer_monthly_cv.mean(),
                    'std': customer_monthly_cv.std(),
                    'distribution': customer_monthly_cv.describe()
                }
            else:
                print("    ⚠️ month 컬럼이 없어 월간 변동계수 계산 건너뛰기")
                variability_metrics['monthly_cv'] = {'mean': np.nan, 'std': np.nan}
        except Exception as e:
            print(f"    ⚠️ 월간 변동계수 계산 실패: {e}")
            variability_metrics['monthly_cv'] = {'mean': np.nan, 'std': np.nan}
        
        # 4. 추가 변동성 지표
        # 범위 기반 변동성
        try:
            customer_range_cv = data.groupby('customer_id').apply(
                lambda x: (x['daily_mean'].max() - x['daily_mean'].min()) / x['daily_mean'].mean() if x['daily_mean'].mean() > 0 else np.nan
            )
            
            variability_metrics['range_based_cv'] = {
                'mean': customer_range_cv.mean(),
                'std': customer_range_cv.std(),
                'distribution': customer_range_cv.describe()
            }
        except Exception as e:
            print(f"    ⚠️ 범위 기반 변동계수 계산 실패: {e}")
            variability_metrics['range_based_cv'] = {'mean': np.nan, 'std': np.nan}
        
        print(f"  ✅ 기본 변동성 지표 계산 완료")
        return variability_metrics
    
    def _analyze_variability_patterns(self, data):
        """변동성 패턴 분석"""
        print("  🔍 변동성 패턴 분석 중...")
        
        patterns = {}
        
        # 1. 업종별 변동성 비교
        usage_purpose_col = None
        possible_usage_cols = ['사용용도', 'usage_purpose', 'Usage_Purpose']
        
        for col in possible_usage_cols:
            if col in data.columns:
                usage_purpose_col = col
                break
        
        if usage_purpose_col:
            try:
                industry_variability = data.groupby([usage_purpose_col, 'customer_id']).apply(
                    lambda x: x['daily_mean'].std() / x['daily_mean'].mean() if x['daily_mean'].mean() > 0 else np.nan
                ).reset_index()
                industry_variability.columns = [usage_purpose_col, 'customer_id', 'cv']
                
                industry_cv_summary = industry_variability.groupby(usage_purpose_col)['cv'].agg(['mean', 'std', 'count'])
                patterns['industry_variability'] = industry_cv_summary
                print(f"  ✅ {usage_purpose_col} 기준 업종별 변동성 분석 완료")
            except Exception as e:
                print(f"  ⚠️ 업종별 변동성 분석 실패: {e}")
        
        # 2. 계절별 변동성 차이
        try:
            if 'month' in data.columns:
                seasonal_variability = data.groupby(['customer_id', 'month']).apply(
                    lambda x: x['daily_mean'].std() / x['daily_mean'].mean() if x['daily_mean'].mean() > 0 else np.nan
                ).reset_index()
                seasonal_variability.columns = ['customer_id', 'month', 'cv']
                
                seasonal_cv_summary = seasonal_variability.groupby('month')['cv'].agg(['mean', 'std', 'count'])
                patterns['seasonal_variability'] = seasonal_cv_summary
                print(f"  ✅ 계절별 변동성 분석 완료")
            else:
                print("  ⚠️ month 컬럼이 없어 계절별 변동성 분석 건너뛰기")
        except Exception as e:
            print(f"  ⚠️ 계절별 변동성 분석 실패: {e}")
        
        # 3. 사용량 규모별 변동성
        try:
            customer_avg_usage = data.groupby('customer_id')['daily_mean'].mean()
            customer_cv = data.groupby('customer_id').apply(
                lambda x: x['daily_mean'].std() / x['daily_mean'].mean() if x['daily_mean'].mean() > 0 else np.nan
            )
            
            # 사용량 규모별 그룹핑
            usage_quantiles = customer_avg_usage.quantile([0.33, 0.67])
            def categorize_usage(usage):
                if usage <= usage_quantiles.iloc[0]:
                    return 'Low'
                elif usage <= usage_quantiles.iloc[1]:
                    return 'Medium'
                else:
                    return 'High'
            
            customer_usage_category = customer_avg_usage.apply(categorize_usage)
            usage_cv_df = pd.DataFrame({
                'usage_category': customer_usage_category,
                'cv': customer_cv
            })
            
            usage_cv_summary = usage_cv_df.groupby('usage_category')['cv'].agg(['mean', 'std', 'count'])
            patterns['usage_level_variability'] = usage_cv_summary
            print(f"  ✅ 사용량 규모별 변동성 분석 완료")
        except Exception as e:
            print(f"  ⚠️ 사용량 규모별 변동성 분석 실패: {e}")
        
        print(f"  ✅ 변동성 패턴 분석 완료 ({len(patterns)}개 패턴)")
        return patterns
    
    # ============ 4단계: 이상 패턴 탐지 (60분) ============
    
    def detect_anomalous_patterns(self, processed_data):
        """
        이상 패턴 탐지
        """
        print("\n🎯 4단계: 이상 패턴 탐지 시작...")
        
        # 통계적 이상치 식별
        statistical_outliers = self._identify_statistical_outliers(processed_data)
        
        # 시계열 이상치 탐지
        temporal_anomalies = self._detect_temporal_anomalies(processed_data)
        
        # 비정상 패턴 정의
        abnormal_patterns = self._define_abnormal_patterns(processed_data)
        
        anomaly_results = {
            'statistical_outliers': statistical_outliers,
            'temporal_anomalies': temporal_anomalies,
            'abnormal_patterns': abnormal_patterns
        }
        
        return anomaly_results
    
    def _identify_statistical_outliers(self, data):
        """통계적 이상치 식별"""
        print("  🔍 통계적 이상치 식별 중...")
        
        outliers = {}
        
        # IQR 방법
        Q1 = data['daily_mean'].quantile(0.25)
        Q3 = data['daily_mean'].quantile(0.75)
        IQR = Q3 - Q1
        
        iqr_outliers = data[
            (data['daily_mean'] < Q1 - 1.5 * IQR) | 
            (data['daily_mean'] > Q3 + 1.5 * IQR)
        ]
        
        outliers['iqr_outliers'] = {
            'count': len(iqr_outliers),
            'rate': len(iqr_outliers) / len(data) * 100,
            'customer_count': iqr_outliers['customer_id'].nunique()
        }
        
        # Z-score 방법
        z_scores = np.abs((data['daily_mean'] - data['daily_mean'].mean()) / data['daily_mean'].std())
        zscore_outliers = data[z_scores > 3]
        
        outliers['zscore_outliers'] = {
            'count': len(zscore_outliers),
            'rate': len(zscore_outliers) / len(data) * 100,
            'customer_count': zscore_outliers['customer_id'].nunique()
        }
        
        print(f"  ✅ IQR 이상치: {outliers['iqr_outliers']['count']:,}건 ({outliers['iqr_outliers']['rate']:.2f}%)")
        print(f"  ✅ Z-score 이상치: {outliers['zscore_outliers']['count']:,}건 ({outliers['zscore_outliers']['rate']:.2f}%)")
        
        return outliers
    
    def _detect_temporal_anomalies(self, data):
        """시계열 이상치 탐지"""
        print("  ⏰ 시계열 이상치 탐지 중...")
        
        temporal_anomalies = {}
        
        # 고객별 시계열 이상치 탐지
        for customer_id in data['customer_id'].unique()[:100]:  # 샘플로 100명만
            customer_data = data[data['customer_id'] == customer_id].sort_values('date')
            
            if len(customer_data) < 30:  # 최소 30일 데이터 필요
                continue
            
            # 급격한 증가/감소 탐지 (>200% 변화)
            customer_data['pct_change'] = customer_data['daily_mean'].pct_change()
            sudden_changes = customer_data[abs(customer_data['pct_change']) > 2.0]  # 200% 변화
            
            # 연속적인 0값 탐지
            zero_streaks = customer_data[customer_data['daily_mean'] == 0]
            
            if len(sudden_changes) > 0 or len(zero_streaks) > 5:  # 5일 이상 연속 0값
                temporal_anomalies[customer_id] = {
                    'sudden_changes': len(sudden_changes),
                    'zero_streaks': len(zero_streaks)
                }
        
        print(f"  ✅ {len(temporal_anomalies):,}명 고객에서 시계열 이상 탐지")
        
        return temporal_anomalies
    
    def _define_abnormal_patterns(self, data):
        """비정상 패턴 정의"""
        print("  📋 비정상 패턴 정의 중...")
        
        abnormal_patterns = {
            'pattern_definitions': {
                1: '전력 사용 급증/급감 (사업 확장/축소)',
                2: '사용 패턴 변화 (운영시간 변경)', 
                3: '효율성 급변 (설비 교체/고장)',
                4: '계절성 이탈 (사업 모델 변화)'
            },
            'detection_criteria': {
                'usage_spike': 'daily_mean > mean + 3*std',
                'usage_drop': 'daily_mean < mean - 3*std',
                'pattern_shift': 'monthly pattern change > 50%',
                'efficiency_change': 'weekly efficiency variance > threshold'
            }
        }
        
        print(f"  ✅ {len(abnormal_patterns['pattern_definitions'])}가지 비정상 패턴 정의 완료")
        
        return abnormal_patterns
    
    # ============ 5단계: 전처리 방향 결정 (30분) ============
    
    def decide_preprocessing_strategy(self, data_quality_report, pattern_analysis, variability_analysis):
        """
        전처리 방향 결정
        """
        print("\n🔧 5단계: 전처리 방향 결정...")
        
        preprocessing_strategy = {
            'missing_data_handling': self._decide_missing_data_strategy(data_quality_report),
            'outlier_handling': self._decide_outlier_strategy(data_quality_report),
            'normalization_method': self._decide_normalization_strategy(pattern_analysis),
            'feature_engineering': self._decide_feature_engineering(pattern_analysis, variability_analysis)
        }
        
        self._print_preprocessing_summary(preprocessing_strategy)
        
        return preprocessing_strategy
    
    def _decide_missing_data_strategy(self, quality_report):
        """결측치 처리 전략 결정"""
        completeness_rate = quality_report['data_completeness']['completeness_rate']
        
        if completeness_rate > 95:
            strategy = "선형보간 또는 forward fill"
        elif completeness_rate > 80:
            strategy = "계절성 고려 보간"
        else:
            strategy = "장기 결측 기간 분석 제외"
        
        return {
            'completeness_rate': completeness_rate,
            'recommended_strategy': strategy
        }
    
    def _decide_outlier_strategy(self, quality_report):
        """이상치 처리 전략 결정"""
        outlier_rate = quality_report['anomaly_detection']['outlier_rate_iqr']
        
        if outlier_rate < 1:
            strategy = "이상치 유지 (정상 범위)"
        elif outlier_rate < 5:
            strategy = "extreme outlier만 제거"
        else:
            strategy = "robust 통계량 사용"
        
        return {
            'outlier_rate': outlier_rate,
            'recommended_strategy': strategy
        }
    
    def _decide_normalization_strategy(self, pattern_analysis):
        """정규화 방법 결정"""
        customer_stats = pattern_analysis['customer_segmentation']
        usage_std = customer_stats['usage_distribution']['std']
        usage_mean = customer_stats['usage_distribution']['mean']
        cv = usage_std / usage_mean if usage_mean > 0 else 0
        
        if cv > 1.0:
            strategy = "고객별 표준화 + 로그 변환"
        elif cv > 0.5:
            strategy = "고객별 표준화"
        else:
            strategy = "전체 Min-Max 정규화"
        
        return {
            'coefficient_of_variation': cv,
            'recommended_strategy': strategy
        }
    
    def _decide_feature_engineering(self, pattern_analysis, variability_analysis):
        """피처 엔지니어링 전략 결정"""
        features_to_create = []
        
        # 시간 기반 피처
        time_patterns = pattern_analysis['time_patterns']
        if 'monthly' in time_patterns:
            features_to_create.extend([
                'month_sin', 'month_cos',  # 계절 순환 피처
                'is_summer', 'is_winter',  # 계절 더미 변수
            ])
        
        # 요일 기반 피처
        if 'weekday' in time_patterns:
            features_to_create.extend([
                'weekday_sin', 'weekday_cos',  # 요일 순환 피처
                'is_weekend'                   # 주말 여부
            ])
        
        # 변동성 기반 피처
        if variability_analysis:
            features_to_create.extend([
                'rolling_mean_7d',       # 7일 이동평균
                'rolling_std_7d',        # 7일 이동표준편차
                'usage_volatility',      # 변동성 지수
            ])
        
        # 고객 기반 피처
        features_to_create.extend([
            'customer_avg_usage',      # 고객 평균 사용량
            'customer_usage_rank',     # 고객 사용량 순위
            'deviation_from_avg'       # 평균 대비 편차
        ])
        
        return {
            'features_to_create': features_to_create,
            'total_features': len(features_to_create)
        }
    
    def _print_preprocessing_summary(self, strategy):
        """전처리 전략 요약 출력"""
        print("\n" + "="*60)
        print("🔧 전처리 전략 결정 완료")
        print("="*60)
        
        print(f"📋 결측치 처리: {strategy['missing_data_handling']['recommended_strategy']}")
        print(f"🎯 이상치 처리: {strategy['outlier_handling']['recommended_strategy']}")
        print(f"📊 정규화 방법: {strategy['normalization_method']['recommended_strategy']}")
        print(f"🛠️ 생성할 피처: {strategy['feature_engineering']['total_features']}개")
        
        print("\n💡 변동계수 설계를 위한 인사이트:")
        print("- 어떤 변동성 지표가 실제 사업 변화를 잘 반영하는가?")
        print("- 업종별로 다른 임계값이 필요한가?")
        print("- 시간 윈도우는 얼마나 설정해야 하는가?")
        print("- 계절성 보정이 필요한가?")
    
    # ============ 시각화 및 리포트 생성 ============
    
    def create_eda_visualizations(self, processed_data):
        """탐색적 데이터 분석 시각화"""
        print("\n📈 EDA 시각화 생성 중...")
        
        # 1. 요일별 사용 패턴 (시간대별 대신)
        self._plot_weekday_patterns_daily(processed_data)
        
        # 2. 요일별 사용 패턴 (바 차트)
        self._plot_weekday_patterns(processed_data)
        
        # 3. 월별 사용량 박스플롯
        self._plot_monthly_boxplot(processed_data)
        
        # 4. 고객별 사용량 분포
        self._plot_customer_distribution(processed_data)
        
        print("✅ 시각화 생성 완료")
    
    def _plot_weekday_patterns_daily(self, data):
        """요일별 평균 사용 패턴 (라인 차트)"""
        if 'weekday' in data.columns:
            weekday_avg = data.groupby('weekday')['daily_mean'].mean()
            
            plt.figure(figsize=(10, 6))
            plt.plot(range(7), weekday_avg.values, marker='o', linewidth=2, markersize=8)
            plt.title('Daily Power Usage by Weekday', fontsize=14, fontweight='bold')
            plt.xlabel('Weekday')
            plt.ylabel('Average Usage (kWh)')
            plt.xticks(range(7), ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
        else:
            print("  ⚠️ 요일별 패턴 시각화 불가 - weekday 컬럼 없음")
    
    def _plot_weekday_patterns(self, data):
        """요일별 사용 패턴"""
        if 'weekday' not in data.columns:
            print("  ⚠️ 요일별 패턴 시각화 불가 - weekday 컬럼 없음")
            return
            
        weekday_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        weekday_avg = data.groupby('weekday')['daily_mean'].mean()
        
        plt.figure(figsize=(10, 6))
        bars = plt.bar(range(7), weekday_avg.values, color=['skyblue' if i < 5 else 'lightcoral' for i in range(7)])
        plt.title('Average Power Usage by Weekday', fontsize=14, fontweight='bold')
        plt.xlabel('Weekday')
        plt.ylabel('Average Usage (kWh)')
        plt.xticks(range(7), weekday_names)
        plt.grid(True, alpha=0.3, axis='y')
        
        # 주중/주말 구분 표시
        for i, bar in enumerate(bars):
            if i >= 5:  # 주말
                bar.set_label('Weekend' if i == 5 else '')
            else:  # 주중
                bar.set_label('Weekday' if i == 0 else '')
        
        plt.legend()
        plt.tight_layout()
        plt.show()
    
    def _plot_monthly_boxplot(self, data):
        """월별 사용량 박스플롯"""
        if 'month' not in data.columns:
            print("  ⚠️ 월별 패턴 시각화 불가 - month 컬럼 없음")
            return
            
        plt.figure(figsize=(14, 8))
        
        # 월별 데이터 준비
        monthly_data = [data[data['month'] == m]['daily_mean'].values for m in range(1, 13)]
        month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                      'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        
        # 데이터가 있는 월만 표시
        valid_months = []
        valid_data = []
        valid_names = []
        
        for i, month_data in enumerate(monthly_data):
            if len(month_data) > 0:
                valid_months.append(i + 1)
                valid_data.append(month_data)
                valid_names.append(month_names[i])
        
        if not valid_data:
            print("  ⚠️ 월별 데이터 없음")
            return
        
        box_plot = plt.boxplot(valid_data, labels=valid_names, patch_artist=True)
        
        # 계절별 색상 구분
        colors = []
        for month in valid_months:
            if month in [12, 1, 2]:  # 겨울
                colors.append('lightblue')
            elif month in [3, 4, 5]:  # 봄
                colors.append('lightgreen')
            elif month in [6, 7, 8]:  # 여름
                colors.append('lightcoral')
            else:  # 가을
                colors.append('orange')
        
        for patch, color in zip(box_plot['boxes'], colors):
            patch.set_facecolor(color)
            patch.set_alpha(0.7)
        
        plt.title('Monthly Power Usage Distribution', fontsize=14, fontweight='bold')
        plt.xlabel('Month')
        plt.ylabel('Daily Average Usage (kWh)')
        plt.grid(True, alpha=0.3, axis='y')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    
    def _plot_customer_distribution(self, data):
        """고객별 평균 사용량 분포"""
        customer_avg = data.groupby('customer_id')['daily_mean'].mean()
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # 히스토그램
        ax1.hist(customer_avg.values, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
        ax1.set_title('Customer Average Usage Distribution', fontweight='bold')
        ax1.set_xlabel('Average Usage (kWh)')
        ax1.set_ylabel('Number of Customers')
        ax1.grid(True, alpha=0.3)
        
        # 박스플롯
        ax2.boxplot(customer_avg.values, vert=True)
        ax2.set_title('Customer Average Usage Boxplot', fontweight='bold')
        ax2.set_ylabel('Average Usage (kWh)')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def generate_eda_report(self):
        """EDA 종합 리포트 생성"""
        print("\n" + "="*70)
        print("KEPCO LP Data Preprocessing and EDA Report")
        print("="*70)
        
        if hasattr(self, 'data_quality_report'):
            print("\n🔍 Step 1: Data Quality Check Results")
            print("-" * 40)
            quality = self.data_quality_report
            print(f"Total Records: {quality['lp_quality']['total_records']:,}")
            print(f"Customers: {quality['lp_quality']['unique_customers']:,}")
            print(f"Data Completeness: {quality['data_completeness']['completeness_rate']:.2f}%")
            print(f"Outlier Rate: {quality['anomaly_detection']['outlier_rate_iqr']:.3f}%")
        
        if hasattr(self, 'pattern_analysis'):
            print("\n📊 Step 2: Basic Pattern Analysis Results")
            print("-" * 40)
            pattern = self.pattern_analysis
            if 'customer_segmentation' in pattern:
                seg = pattern['customer_segmentation']
                print(f"Analyzed Customers: {seg['customer_count']:,}")
                print(f"Average Usage: {seg['usage_distribution']['mean']:.2f} kWh")
                print(f"Usage Std Dev: {seg['usage_distribution']['std']:.2f} kWh")
        
        if hasattr(self, 'variability_analysis'):
            print("\n📈 Step 3: Variability Analysis Results")
            print("-" * 40)
            var = self.variability_analysis
            if 'basic_variability' in var:
                basic = var['basic_variability']
                print(f"Average Daily CV: {basic['daily_cv']['mean']:.4f}")
                if not pd.isna(basic['weekly_cv']['mean']):
                    print(f"Average Weekly CV: {basic['weekly_cv']['mean']:.4f}")
                if not pd.isna(basic['monthly_cv']['mean']):
                    print(f"Average Monthly CV: {basic['monthly_cv']['mean']:.4f}")
        
        print("\n💡 Next Steps Recommendations:")
        print("- Define and design variability coefficient")
        print("- Implement stacking ensemble model") 
        print("- Apply overfitting prevention techniques")
        print("- Develop business activity change prediction algorithm")
        
        print("\n" + "="*70)

# 사용 예시 - LP 데이터 파일들을 결합하여 분석

# 데이터 로딩 (여러 LP 파일들 결합)
lp_files = ['LP데이터1.csv', 'LP데이터2.csv']  # 한 달을 반으로 나눈 파일들
customer_data = pd.read_excel('고객번호.xlsx')
weather_data = pd.read_csv('weather_daily_processed.csv') 
calendar_data = pd.read_csv('power_analysis_calendar_2022_2025.csv')

# 전처리 및 EDA 실행
preprocessor = KepcoDataPreprocessor()

# LP 데이터 결합
combined_lp_data = preprocessor.load_and_combine_lp_data(lp_files)

# 1단계: 데이터 품질 점검 (30분)
quality_report = preprocessor.check_data_quality(combined_lp_data, customer_data)

# 2단계: 기본 패턴 탐색 (60분) - 기상 데이터도 함께 병합!
pattern_analysis = preprocessor.analyze_basic_patterns(combined_lp_data, customer_data, weather_data)

# 3단계: 변동성 기초 분석 (90분)
variability_analysis = preprocessor.analyze_variability(pattern_analysis['processed_data'])

# 4단계: 이상 패턴 탐지 (60분)
anomaly_results = preprocessor.detect_anomalous_patterns(pattern_analysis['processed_data'])

# 5단계: 전처리 방향 결정 (30분)
preprocessing_strategy = preprocessor.decide_preprocessing_strategy(
    quality_report, pattern_analysis, variability_analysis
)

# 시각화 생성 (폰트 오류 없이)
preprocessor.create_eda_visualizations(pattern_analysis['processed_data'])

# 종합 리포트 생성
preprocessor.generate_eda_report()

print("✅ Complete LP data preprocessing and EDA finished!")
print("📤 Next: Define variability coefficient and implement stacking model")


In [ ]:
# 오류 수정된 실행 코드
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
import os
warnings.filterwarnings('ignore')

class KEPCOVolatilityAnalyzer:
    def __init__(self):
        self.lp_data = None
        self.weather_data = None
        self.calendar_data = None
        self.customer_data = None
        self.combined_data = None
        self.volatility_metrics = {}
        
    def load_real_data(self, lp_files=None, weather_file='weather_daily_processed.csv', 
                      calendar_file='power_analysis_calendar_2022_2025.csv', 
                      customer_file='고객번호.xlsx'):
        """실제 파일들을 사용한 데이터 로딩"""
        print("=== 실제 데이터 파일 로딩 및 전처리 ===")
        
        # 현재 작업 디렉토리 확인
        current_dir = os.getcwd()
        print(f"📁 현재 작업 디렉토리: {current_dir}")
        
        # 1. 고객 기본정보 로딩
        try:
            if os.path.exists(customer_file):
                self.customer_data = pd.read_excel(customer_file)
                print(f"✅ 고객 데이터 로딩: {len(self.customer_data):,}명")
            else:
                print(f"⚠️ {customer_file} 파일이 없습니다. 샘플 데이터를 생성합니다.")
                self.customer_data = self._create_sample_customer_data()
        except Exception as e:
            print(f"❌ 고객 데이터 로딩 실패: {e}")
            self.customer_data = self._create_sample_customer_data()
            
        # 2. LP 데이터 로딩 (여러 파일 통합)
        if lp_files:
            lp_dataframes = []
            for file in lp_files:
                try:
                    if os.path.exists(file):
                        df = pd.read_csv(file)
                        lp_dataframes.append(df)
                        print(f"✅ LP 데이터 로딩: {file} ({len(df):,}건)")
                    else:
                        print(f"⚠️ {file} 파일이 없습니다.")
                except Exception as e:
                    print(f"❌ LP 데이터 로딩 실패: {file} - {e}")
            
            if lp_dataframes:
                self.lp_data = pd.concat(lp_dataframes, ignore_index=True)
                print(f"✅ 전체 LP 데이터: {len(self.lp_data):,}건")
            else:
                print("⚠️ LP 파일 로딩 실패, 샘플 데이터 생성")
                self.lp_data = self._create_test_lp_data()
        else:
            print("⚠️ LP 파일 미지정, 샘플 데이터 생성")
            self.lp_data = self._create_test_lp_data()
            
        # 3. 날씨 데이터 로딩
        try:
            if os.path.exists(weather_file):
                self.weather_data = pd.read_csv(weather_file, encoding='utf-8')
                print(f"✅ 날씨 데이터 로딩: {len(self.weather_data):,}일")
                print(f"   컬럼: {list(self.weather_data.columns)[:5]}...")  # 처음 5개 컬럼만 표시
            else:
                print(f"⚠️ {weather_file} 파일이 없습니다. 샘플 데이터를 생성합니다.")
                self.weather_data = self._create_weather_test_data()
        except Exception as e:
            print(f"❌ 날씨 데이터 로딩 실패: {e}")
            self.weather_data = self._create_weather_test_data()
            
        # 4. 달력 데이터 로딩
        try:
            if os.path.exists(calendar_file):
                self.calendar_data = pd.read_csv(calendar_file, encoding='utf-8')
                print(f"✅ 달력 데이터 로딩: {len(self.calendar_data):,}일")
                print(f"   컬럼: {list(self.calendar_data.columns)[:5]}...")  # 처음 5개 컬럼만 표시
            else:
                print(f"⚠️ {calendar_file} 파일이 없습니다. 샘플 데이터를 생성합니다.")
                self.calendar_data = self._create_calendar_test_data()
        except Exception as e:
            print(f"❌ 달력 데이터 로딩 실패: {e}")
            self.calendar_data = self._create_calendar_test_data()
            
        # 5. 데이터 전처리 및 병합
        self._preprocess_data()
        self.combined_data = self._merge_all_data()
        
        print(f"\n🎯 최종 통합 데이터셋: {len(self.combined_data):,}건")
        return self.combined_data
    
    def _create_sample_customer_data(self):
        """샘플 고객 데이터 생성"""
        customers = []
        for i in range(10):  # 10명 샘플
            customers.append({
                '대체고객번호': f'A{1001+i}',
                '계약전력': np.random.choice([100, 200, 300, 500, 700]),
                '계약종별': np.random.choice([222, 226, 311, 322, 726]),
                '사용용도': np.random.choice(['02', '09']),
                '산업분류': f'업종{np.random.randint(1, 6)}'
            })
        return pd.DataFrame(customers)
    
    def _create_test_lp_data(self):
        """테스트용 LP 데이터 생성"""
        data = []
        customers = [f'A{1001+i}' for i in range(10)]
        start_date = datetime(2024, 3, 1)
        days = 31
        
        for customer in customers:
            # 고객별 고유 특성 (변동성 패턴)
            base_usage = np.random.uniform(30, 150)
            volatility_level = np.random.uniform(0.1, 0.4)
            
            for day in range(days):
                current_date = start_date + timedelta(days=day)
                
                for hour in range(0, 24, 6):  # 6시간 간격
                    for minute in [0, 15, 30, 45]:
                        timestamp = current_date.replace(hour=hour, minute=minute)
                        
                        # 시간대별 패턴
                        if 8 <= hour <= 18:  # 업무시간
                            hour_factor = 1.2
                        elif 19 <= hour <= 22:  # 저녁시간
                            hour_factor = 0.8
                        else:  # 심야시간
                            hour_factor = 0.4
                        
                        # 요일별 패턴
                        weekday = current_date.weekday()
                        if weekday >= 5:  # 주말
                            weekday_factor = 0.6
                        else:  # 평일
                            weekday_factor = 1.0
                        
                        # 최종 전력량 계산
                        power = base_usage * hour_factor * weekday_factor
                        power += np.random.normal(0, base_usage * volatility_level)
                        power = max(5, power)  # 최소값 보장
                        
                        data.append({
                            '대체고객번호': customer,
                            'LP수신일자': timestamp.strftime('%Y-%m-%d-%H:%M'),
                            '순방향유효전력': round(power, 1),
                            '지상무효': round(power * 0.2, 1),
                            '진상무효': round(power * 0.1, 1),
                            '피상전력': round(power * 1.05, 1)
                        })
        
        return pd.DataFrame(data)
    
    def _create_weather_test_data(self):
        """테스트용 날씨 데이터 생성"""
        start_date = datetime(2024, 3, 1)
        days = 31
        
        weather_data = []
        for day in range(days):
            current_date = start_date + timedelta(days=day)
            
            # 3월 기후 특성
            base_temp = 12 + np.random.normal(0, 4)
            temp_range = np.random.uniform(8, 15)
            
            최고기온 = base_temp + temp_range/2
            최저기온 = base_temp - temp_range/2
            평균기온 = (최고기온 + 최저기온) / 2
            
            # 습도 계산
            평균습도 = max(30, min(90, 70 - (평균기온 - 12) * 2 + np.random.normal(0, 10)))
            
            # 강수량
            강수여부 = 1 if np.random.random() < 0.1 else 0
            총강수량 = np.random.exponential(5) if 강수여부 else 0
            
            # 냉난방도일
            냉방도일 = max(0, 평균기온 - 18)
            난방도일 = max(0, 18 - 평균기온)
            
            weather_data.append({
                '날짜': current_date.strftime('%Y-%m-%d'),
                '평균기온': round(평균기온, 1),
                '평균습도': round(평균습도),
                '총강수량': round(총강수량, 1),
                '냉방도일': round(냉방도일, 1),
                '난방도일': round(난방도일, 1),
                '강수여부': 강수여부
            })
        
        return pd.DataFrame(weather_data)
    
    def _create_calendar_test_data(self):
        """테스트용 달력 데이터 생성"""
        start_date = datetime(2024, 3, 1)
        days = 31
        
        calendar_data = []
        for day in range(days):
            current_date = start_date + timedelta(days=day)
            
            weekday = current_date.weekday()
            is_weekend = weekday >= 5
            is_workday = not is_weekend
            is_holiday = (current_date.month == 3 and current_date.day == 1)  # 3.1절
            
            calendar_data.append({
                'date': current_date.strftime('%Y-%m-%d'),
                'year': current_date.year,
                'month': current_date.month,
                'day': current_date.day,
                'weekday': weekday,
                'is_workday': is_workday,
                'is_weekend': is_weekend,
                'is_holiday': is_holiday,
                'workday_indicator': 1 if is_workday and not is_holiday else 0,
                'is_month_start': current_date.day == 1,
                'is_month_end': current_date.day >= 29
            })
        
        return pd.DataFrame(calendar_data)
    
    def _preprocess_data(self):
        """데이터 전처리"""
        # LP 데이터 전처리
        if self.lp_data is not None:
            self.lp_data['datetime'] = pd.to_datetime(self.lp_data['LP수신일자'], format='%Y-%m-%d-%H:%M')
            self.lp_data['date'] = self.lp_data['datetime'].dt.date
            self.lp_data['hour'] = self.lp_data['datetime'].dt.hour
            self.lp_data['weekday'] = self.lp_data['datetime'].dt.weekday
        
        # 날씨 데이터 전처리
        if self.weather_data is not None:
            self.weather_data['날짜'] = pd.to_datetime(self.weather_data['날짜']).dt.date
        
        # 달력 데이터 전처리
        if self.calendar_data is not None:
            self.calendar_data['date'] = pd.to_datetime(self.calendar_data['date']).dt.date
    
    def _merge_all_data(self):
        """모든 데이터 병합"""
        # 일별로 LP 데이터 집계
        daily_lp = self.lp_data.groupby(['대체고객번호', 'date']).agg({
            '순방향유효전력': ['mean', 'std', 'max', 'min', 'sum', 'count']
        }).reset_index()
        
        # 컬럼명 정리
        daily_lp.columns = ['customer_id', 'date', 'power_mean', 'power_std', 
                           'power_max', 'power_min', 'power_sum', 'power_count']
        
        # 변동계수 계산
        daily_lp['daily_cv'] = daily_lp['power_std'] / daily_lp['power_mean']
        daily_lp['daily_cv'] = daily_lp['daily_cv'].fillna(0)
        
        # 날씨 데이터와 병합
        merged = daily_lp.merge(self.weather_data, left_on='date', right_on='날짜', how='left')
        
        # 달력 데이터와 병합
        merged = merged.merge(self.calendar_data, on='date', how='left')
        
        return merged
    
    def calculate_comprehensive_volatility_score(self):
        """종합 변동성 스코어 계산"""
        print("\n=== 종합 변동성 스코어 계산 ===")
        
        if self.combined_data is None:
            print("❌ 먼저 데이터를 로딩해주세요.")
            return None
        
        comprehensive_scores = {}
        
        for customer in self.combined_data['customer_id'].unique():
            customer_data = self.combined_data[self.combined_data['customer_id'] == customer]
            
            if len(customer_data) == 0:
                continue
            
            # 1. 기본 변동계수
            basic_cv = customer_data['daily_cv'].std()
            
            # 2. 날씨 보정 변동계수
            try:
                weather_features = customer_data[['평균기온', '평균습도', '냉방도일', '난방도일']].fillna(0)
                power_target = customer_data['power_mean'].fillna(customer_data['power_mean'].mean())
                
                if len(weather_features) > 1 and len(weather_features) == len(power_target):
                    model = LinearRegression()
                    model.fit(weather_features, power_target)
                    predicted = model.predict(weather_features)
                    residuals = power_target - predicted
                    weather_adjusted_cv = residuals.std() / residuals.mean() if residuals.mean() != 0 else basic_cv
                else:
                    weather_adjusted_cv = basic_cv
            except:
                weather_adjusted_cv = basic_cv
            
            # 3. 달력 보정 변동계수
            try:
                calendar_features = customer_data[['workday_indicator', 'is_month_start', 'is_month_end']].fillna(0)
                
                if len(calendar_features) > 1 and len(calendar_features) == len(power_target):
                    model = LinearRegression()
                    model.fit(calendar_features, power_target)
                    predicted = model.predict(calendar_features)
                    residuals = power_target - predicted
                    calendar_adjusted_cv = residuals.std() / residuals.mean() if residuals.mean() != 0 else basic_cv
                else:
                    calendar_adjusted_cv = basic_cv
            except:
                calendar_adjusted_cv = basic_cv
            
            # 4. 외부 요인 민감도
            temp_correlation = abs(customer_data['평균기온'].corr(customer_data['power_mean'])) or 0
            
            workday_data = customer_data[customer_data['is_workday'] == True]['power_mean']
            weekend_data = customer_data[customer_data['is_weekend'] == True]['power_mean']
            
            if len(workday_data) > 0 and len(weekend_data) > 0:
                weekday_effect = abs(workday_data.mean() - weekend_data.mean())
                weekday_effect = weekday_effect / customer_data['power_mean'].mean() if customer_data['power_mean'].mean() > 0 else 0
            else:
                weekday_effect = 0
            
            # 5. 최종 순수 변동성 (날씨 + 달력 보정)
            pure_volatility = (weather_adjusted_cv + calendar_adjusted_cv) / 2
            
            # 6. 종합 스코어 계산
            weights = {
                'pure_volatility': 0.5,
                'basic_cv': 0.3,
                'temp_sensitivity': 0.1,
                'weekday_effect': 0.1
            }
            
            final_score = (
                weights['pure_volatility'] * pure_volatility +
                weights['basic_cv'] * basic_cv +
                weights['temp_sensitivity'] * temp_correlation +
                weights['weekday_effect'] * weekday_effect
            )
            
            # 7. 영업활동 변화 위험도 평가
            if pure_volatility > 0.3:
                risk_level = "높음"
                recommendation = "영업활동 변화 의심 - 정밀 분석 필요"
            elif pure_volatility > 0.15:
                risk_level = "보통"
                recommendation = "주의 관찰 - 주기적 모니터링"
            else:
                risk_level = "낮음"
                recommendation = "정상 운영 추정 - 안정적"
            
            comprehensive_scores[customer] = {
                'basic_cv': round(basic_cv, 4),
                'weather_adjusted_cv': round(weather_adjusted_cv, 4),
                'calendar_adjusted_cv': round(calendar_adjusted_cv, 4),
                'pure_volatility': round(pure_volatility, 4),
                'temp_sensitivity': round(temp_correlation, 4),
                'weekday_effect': round(weekday_effect, 4),
                'final_score': round(final_score, 4),
                'risk_level': risk_level,
                'recommendation': recommendation,
                'mean_power': round(customer_data['power_mean'].mean(), 1),
                'data_points': len(customer_data)
            }
        
        print("고객번호\t기본CV\t날씨보정CV\t달력보정CV\t순수변동성\t최종점수\t위험도")
        for customer, scores in comprehensive_scores.items():
            print(f"{customer}\t{scores['basic_cv']}\t{scores['weather_adjusted_cv']}\t{scores['calendar_adjusted_cv']}\t{scores['pure_volatility']}\t{scores['final_score']}\t{scores['risk_level']}")
        
        self.volatility_metrics['comprehensive'] = comprehensive_scores
        return comprehensive_scores
    
    def generate_final_report(self):
        """최종 분석 리포트 생성"""
        print("\n" + "="*80)
        print("🏢 한국전력공사 전력 사용패턴 변동계수 개발 - 최종 리포트")
        print("="*80)
        
        if not hasattr(self, 'volatility_metrics') or 'comprehensive' not in self.volatility_metrics:
            comprehensive_scores = self.calculate_comprehensive_volatility_score()
        else:
            comprehensive_scores = self.volatility_metrics['comprehensive']
        
        if not comprehensive_scores:
            print("❌ 변동성 스코어를 먼저 계산해주세요.")
            return None
        
        print("\n📊 분석 개요:")
        print(f"  • 분석 대상: {len(comprehensive_scores)}명 고객")
        print(f"  • 데이터 기간: {self.combined_data['date'].min()} ~ {self.combined_data['date'].max()}")
        print(f"  • 총 데이터 포인트: {len(self.combined_data):,}건")
        
        # 변동성 분포 분석
        pure_volatilities = [s['pure_volatility'] for s in comprehensive_scores.values()]
        final_scores = [s['final_score'] for s in comprehensive_scores.values()]
        
        print(f"\n🎯 순수 변동성 통계:")
        print(f"  • 평균: {np.mean(pure_volatilities):.4f}")
        print(f"  • 표준편차: {np.std(pure_volatilities):.4f}")
        print(f"  • 최소값: {np.min(pure_volatilities):.4f}")
        print(f"  • 최대값: {np.max(pure_volatilities):.4f}")
        
        # 위험도별 분류
        risk_distribution = {}
        for scores in comprehensive_scores.values():
            risk = scores['risk_level']
            risk_distribution[risk] = risk_distribution.get(risk, 0) + 1
        
        print(f"\n🚨 영업활동 변화 위험도 분포:")
        for risk, count in risk_distribution.items():
            percentage = count / len(comprehensive_scores) * 100
            print(f"  • {risk}: {count}명 ({percentage:.1f}%)")
        
        # 상위 위험 고객 식별
        high_risk_customers = {k: v for k, v in comprehensive_scores.items() 
                             if v['risk_level'] == '높음'}
        
        if high_risk_customers:
            print(f"\n⚠️ 고위험 고객 상세 분석:")
            print("고객번호\t순수변동성\t평균전력\t권장사항")
            for customer, scores in sorted(high_risk_customers.items(), 
                                         key=lambda x: x[1]['pure_volatility'], reverse=True):
                print(f"{customer}\t{scores['pure_volatility']:.4f}\t{scores['mean_power']}kW\t{scores['recommendation']}")
        
        print(f"\n💡 알고리즘 특징:")
        print("  ✅ 다차원 변동성 지표: 기본/날씨보정/달력보정/순수변동성")
        print("  ✅ 외부요인 제거: 날씨 및 달력 효과 자동 보정")
        print("  ✅ 위험도 자동분류: 3단계 위험도 + 맞춤 권장사항")
        print("  ✅ 실시간 모니터링: 영업활동 변화 조기 감지")
        print("  ✅ 확장 가능성: 추가 외부요인 쉽게 통합 가능")
        
        return {
            'comprehensive_scores': comprehensive_scores,
            'statistics': {
                'mean_pure_volatility': np.mean(pure_volatilities),
                'std_pure_volatility': np.std(pure_volatilities),
                'risk_distribution': risk_distribution
            },
            'high_risk_customers': high_risk_customers
        }

def main():
    """메인 실행 함수 - 오류 수정 버전"""
    print("🚀 한국전력공사 전력 사용패턴 변동계수 분석 실행")
    print("="*70)
    
    # 분석기 초기화
    analyzer = KEPCOVolatilityAnalyzer()
    
    # 실제 파일들로 데이터 로딩
    try:
        print("📁 실제 데이터 파일 로딩 시도...")
        
        # 실제 파일 경로 설정
        lp_files = ['LP데이터1.csv', 'LP데이터2.csv']
        weather_file = 'weather_daily_processed.csv'
        calendar_file = 'power_analysis_calendar_2022_2025.csv'
        customer_file = '고객번호.xlsx'
        
        # 데이터 로딩
        combined_data = analyzer.load_real_data(
            lp_files=lp_files,
            weather_file=weather_file,
            calendar_file=calendar_file,
            customer_file=customer_file
        )
        
        print(f"✅ 데이터 로딩 완료: {len(combined_data):,}건")
        
    except Exception as e:
        print(f"⚠️ 실제 파일 로딩 실패: {e}")
        print("📊 샘플 데이터로 분석을 진행합니다...")
        
        # 샘플 데이터로 대체
        combined_data = analyzer.load_real_data()
    
    # 종합 변동성 스코어 계산
    print("\n🔄 종합 변동성 분석 시작...")
    volatility_scores = analyzer.calculate_comprehensive_volatility_score()
    
    # 최종 리포트 생성
    print("\n📊 최종 리포트 생성...")
    final_report = analyzer.generate_final_report()
    
    # 결과 저장 (오류 수정)
    if volatility_scores:
        try:
            # 현재 디렉토리에 파일 저장 (영어 파일명 사용)
            results_df = pd.DataFrame(volatility_scores).T
            
            # 저장 경로 확인
            output_file = 'volatility_analysis_result.csv'
            results_df.to_csv(output_file, encoding='utf-8-sig')
            print(f"💾 결과가 '{output_file}'에 저장되었습니다.")
            
            # 추가 요약 파일도 저장
            summary_file = 'analysis_summary.txt'
            with open(summary_file, 'w', encoding='utf-8') as f:
                f.write("한국전력공사 전력 사용패턴 변동계수 분석 요약\n")
                f.write("="*50 + "\n")
                f.write(f"분석 고객 수: {len(volatility_scores)}명\n")
                f.write(f"분석 완료 시간: {datetime.now()}\n")
                
                # 위험도별 분포
                risk_dist = {}
                for scores in volatility_scores.values():
                    risk = scores['risk_level']
                    risk_dist[risk] = risk_dist.get(risk, 0) + 1
                    
                f.write("\n위험도 분포:\n")
                for risk, count in risk_dist.items():
                    f.write(f"  {risk}: {count}명\n")
                    
                # 고위험 고객
                high_risk = {k: v for k, v in volatility_scores.items() if v['risk_level'] == '높음'}
                if high_risk:
                    f.write(f"\n고위험 고객 ({len(high_risk)}명):\n")
                    for customer, scores in high_risk.items():
                        f.write(f"  {customer}: 순수변동성 {scores['pure_volatility']:.4f}\n")
            
            print(f"📄 요약 리포트가 '{summary_file}'에 저장되었습니다.")
            
        except Exception as save_error:
            print(f"⚠️ 파일 저장 중 오류 발생: {save_error}")
            print("📊 결과를 콘솔에 출력합니다:")
            
            # 콘솔에 결과 출력
            print("\n=== 분석 결과 ===")
            for customer, scores in volatility_scores.items():
                print(f"{customer}: 위험도 {scores['risk_level']}, 순수변동성 {scores['pure_volatility']:.4f}")
        
        # 요약 통계
        print("\n📈 분석 요약:")
        print(f"  • 분석 고객 수: {len(volatility_scores)}명")
        
        # 위험도별 분포
        risk_dist = {}
        for scores in volatility_scores.values():
            risk = scores['risk_level']
            risk_dist[risk] = risk_dist.get(risk, 0) + 1
            
        print("  • 위험도 분포:")
        for risk, count in risk_dist.items():
            percentage = count / len(volatility_scores) * 100
            print(f"    - {risk}: {count}명 ({percentage:.1f}%)")
            
        # 고위험 고객 출력
        high_risk = {k: v for k, v in volatility_scores.items() if v['risk_level'] == '높음'}
        if high_risk:
            print(f"\n⚠️ 고위험 고객 ({len(high_risk)}명):")
            for customer, scores in sorted(high_risk.items(), 
                                         key=lambda x: x[1]['pure_volatility'], reverse=True):
                print(f"  {customer}: 순수변동성 {scores['pure_volatility']:.4f} - {scores['recommendation']}")
        else:
            print("\n✅ 고위험 고객이 발견되지 않았습니다.")
    
    print("\n🎯 분석 완료!")
    return final_report

if __name__ == "__main__":
    # 실행
    try:
        report = main()
        
        # 추가 분석 안내
        print("\n💡 추가 분석 옵션:")
        print("1. 시각화 생성")
        print("2. 상세 고객 분석")
        print("3. 예측 모델 구축")
        print("4. 실시간 모니터링 설정")
        
    except Exception as main_error:
        print(f"❌ 실행 중 오류 발생: {main_error}")
        print("🔧 오류 해결 방법:")
        print("1. 파일 경로 확인")
        print("2. 필요한 라이브러리 설치 확인")
        print("3. 데이터 파일 형식 확인")
        
        # 간단한 디버깅 정보
        import traceback
        print("\n🐛 상세 오류 정보:")
        traceback.print_exc()